In [66]:
#DATASET :
#https://www.kaggle.com/c/fake-news/data

In [67]:
#APPROACH  PRETRAINED 
# we can first separately learn word embeddings and then pass to the embedding layer.
#This approach also allows to use any pre-trained word embedding
#and also saves the time in training the classification model.

#We will use the Gensim implementation of Word2Vec

In [68]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [69]:
#Check if null columns are there  and delete the rows with null values 
print("current shape is ", df.shape)
print(df.isnull().sum())
#drop null values
df= df.dropna()
#reset index as some rows got deleted
df.reset_index(inplace=True)
print("current shape is ", df.shape)

current shape is  (20800, 5)
id           0
title      558
author    1957
text        39
label        0
dtype: int64
current shape is  (18285, 6)


In [70]:
#Getting X and y
#x is every column except label 
X=df.drop('label',axis=1)
y=df['label']

In [71]:
#make a copy of df 
messages = df.copy()
#see 2nd row of title column 
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [72]:
#text pre processing 


In [73]:
#import required library 
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

In [74]:
def clean_text(df):
    all_reviews = list()
    lines = df["title"].values.tolist()
    for text in lines:
        text = text.lower()
        text = re.sub('[^a-zA-Z]', ' ', text)
        words=nltk.word_tokenize(text)
        stop_words= set(stopwords.words('english'))
        stop_words.discard("not")
        words_without_stop_words=[word for word in words if word not in stop_words]
        #words=[lemmatizer.lemmatize(word) for word in words_without_stop_words ]
        words=[ps.stem(word) for word in words_without_stop_words ]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

corpus = clean_text(df)

In [75]:
print("length of corpus is means total number of sentences are :", len(corpus))
corpus[0:5]


length of corpus is means total number of sentences are : 18285


['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [76]:
import tensorflow as tf 
tf.__version__

'2.0.0'

In [77]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences 
#Word Embedding is a representation of text where words that have the same meaning have a similar representation.Embedding layers 
#hrlps to represnts text in vector format 
#pad sequences to make vector of equal length if vector is not that much length then 0s will be added 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.preprocessing.text import one_hot 
#one_hot to make one hot representation of vector 


In [78]:
#define vocabulary size  to be cleared later 
voc_size =5000
print(len(corpus))
#generally voc_size is kept 30%more than len(corpus) but here it is not considered


18285


In [79]:
#onehotRepresentation
#means each word will have distinct unique numbers based on hashing 
onehot_repr =[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1350, 3615, 3033, 4082, 451, 1441, 3627, 4004, 1453, 3947],
 [2725, 2475, 3986, 1236, 3227, 1258, 70],
 [3824, 305, 4868, 4141],
 [436, 1709, 2687, 1691, 682, 2041],
 [3754, 3227, 4418, 4126, 3755, 2011, 3227, 148, 1948, 963],
 [1725,
  651,
  1878,
  4973,
  2319,
  4277,
  1144,
  610,
  1294,
  224,
  1145,
  3776,
  2507,
  3764,
  70],
 [4773, 476, 1152, 1097, 1495, 665, 3617, 2826, 3867, 1063, 903],
 [740, 2280, 1452, 4198, 2325, 4617, 4277, 929, 3867, 1063, 903],
 [3662, 2465, 1705, 2771, 4003, 3114, 3843, 1878, 4277, 4424],
 [1745, 2853, 2515, 3579, 4434, 1017, 1340, 3614],
 [4565, 471, 1815, 3589, 2113, 4202, 3783, 2970, 3693, 4201, 1867],
 [1691, 1489, 451, 3114, 4277, 2325],
 [2449, 79, 1653, 4176, 1118, 4341, 3364, 2223, 2286],
 [101, 3996, 3491, 4310, 3721, 1474, 3727, 3867, 1063, 903],
 [4091, 4964, 696, 1992, 504, 3867, 1063, 903],
 [3977, 2181, 1006, 1571, 1301, 2414, 4179, 927, 3254, 1817],
 [3036, 2114, 2475],
 [1821, 4763, 2290, 664, 4277, 3382, 1884, 70],
 [1628, 

In [80]:
#embeding layer 
#In the above onehot_repr we can see that length of statements are not same 
#before processing through embedding layer we need to make every sentences the same size 
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4004 1453 3947]
 [   0    0    0 ... 3227 1258   70]
 [   0    0    0 ...  305 4868 4141]
 ...
 [   0    0    0 ... 3867 1063  903]
 [   0    0    0 ... 4207 2026 3134]
 [   0    0    0 ... 3439 3129 3551]]


In [81]:
#total number od sentences 
len(embedded_docs)

18285

In [82]:
#see the first line after padding it will be of length 20
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1350,
       3615, 3033, 4082,  451, 1441, 3627, 4004, 1453, 3947])

In [83]:
#LSTM Model creation
embedding_vector_features=40
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metric='accuracy')
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [84]:
#reassign x and y
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [85]:
print("Shape of X", X_final.shape)
print("Shape of y", y_final.shape)

Shape of X (18285, 20)
Shape of y (18285,)


In [88]:
#divide the data into train and test split
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 10s 851us/sample - loss: 0.3480 - val_loss: 0.1954
Epoch 2/10
12250/12250 [==============================] - 7s 589us/sample - loss: 0.1440 - val_loss: 0.2039
Epoch 3/10
12250/12250 [==============================] - 7s 586us/sample - loss: 0.1042 - val_loss: 0.2117
Epoch 4/10
12250/12250 [==============================] - 8s 659us/sample - loss: 0.0758 - val_loss: 0.2707
Epoch 5/10
12250/12250 [==============================] - 7s 595us/sample - loss: 0.0594 - val_loss: 0.2631
Epoch 6/10
12250/12250 [==============================] - 8s 621us/sample - loss: 0.0392 - val_loss: 0.4098
Epoch 7/10
12250/12250 [==============================] - 8s 676us/sample - loss: 0.0277 - val_loss: 0.3641
Epoch 8/10
12250/12250 [==============================] - 8s 679us/sample - loss: 0.0195 - val_loss: 0.5293
Epoch 9/10
12250/12250 [==============================] - 9s 722us/sample - loss: 0.01

In [90]:
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3165,  254],
       [ 317, 2299]], dtype=int64)

In [91]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9053852526926264